In [1]:
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('datos/properati.csv')
dataframe = pycsv.csvToDataFrame(sqlCtx, plaintext_rdd)
data = dataframe.rdd

In [2]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [3]:
def recalcular_precios(row):
    precio_aprox_usd = None
    precio_aprox_local = None
    precio_por_m2 = None
    precio_usd_por_m2 = None

    if (row.currency == 'USD'):
        precio_aprox_usd = float(row.price)
        precio_aprox_local = precio_aprox_usd * 17.6445
        
    if (row.currency == 'ARS'):
        precio_aprox_local = float(row.price)
        precio_aprox_usd = precio_aprox_local // 17.6445
        
    if (row.surface_total_in_m2 and precio_aprox_local and precio_aprox_usd):
        precio_por_m2 = precio_aprox_local // row.surface_total_in_m2
        precio_usd_por_m2 = precio_aprox_usd // row.surface_total_in_m2
        
    return(row.created_on, row.operation, row.property_type, row.place_name, row.place_with_parent_names,\
           row.country_name, row.state_name, row.geonames_id, row.lat, row.lon, row.price, row.currency,\
           precio_aprox_local, precio_aprox_usd, row.surface_total_in_m2, row.surface_covered_in_m2, precio_usd_por_m2,\
           precio_por_m2, row.floor, row.rooms, row.expenses, row.description)

In [4]:
nuevo=data.map(recalcular_precios)
nuevo.toDF().toPandas().to_csv('datos/properati_remap.csv',header=['created_on','operation','property_type','place_name','place_with_parent_names','country_name','state_name','geonames_id','lat','lon','price','currency','precio_aprox_local','precio_aprox_usd','surface_total_in_m2','surface_covered_in_m2','precio_usd_por_m2','precio_por_m2','floor','rooms','expenses','description'])